In [62]:
import pandas as pd

ROW_LIMIT = 10000
COLUMNS = ["starttime", "stoptime", "start station id", "end station id"]
df = pd.read_csv("data/raw_data.csv", usecols=COLUMNS, nrows=ROW_LIMIT)

In [63]:
df["starttime"] = pd.to_datetime(df["starttime"])
df["stoptime"] = pd.to_datetime(df["stoptime"])

In [64]:
df.dtypes

starttime           datetime64[ns]
stoptime            datetime64[ns]
start station id             int64
end station id               int64
dtype: object

In [32]:
df.head()

,starttime,stoptime,start station id,end station id
0,2015-09-01 00:00:00,2015-09-01 00:04:48,263,307
1,2015-09-01 00:00:00,2015-09-01 00:02:45,495,449
2,2015-09-01 00:00:01,2015-09-01 00:06:08,3119,3118
3,2015-09-01 00:00:07,2015-09-01 00:15:34,536,340
4,2015-09-01 00:00:09,2015-09-01 00:11:07,347,483


In [65]:
# Convert int64index to datetime index
# Group using timegrouper and create new column for start period and stop period

df = df.set_index(pd.DatetimeIndex(df['starttime']))
start_time_group_by = df.groupby(pd.TimeGrouper('5Min'),as_index=False).apply(lambda x: x['starttime'])
df['start_period'] = start_time_group_by.index.get_level_values(0)

df = df.set_index(pd.DatetimeIndex(df['stoptime']))
stop_time_group_by = df.groupby(pd.TimeGrouper('5Min'),as_index=False).apply(lambda x: x['stoptime'])
df['stop_period'] = stop_time_group_by.index.get_level_values(0)

In [122]:
df.head()

,starttime,stoptime,start station id,end station id,start_period,stop_period
2015-09-01 00:04:48,2015-09-01 00:00:00,2015-09-01 00:04:48,263,307,0,0
2015-09-01 00:02:45,2015-09-01 00:00:00,2015-09-01 00:02:45,495,449,0,0
2015-09-01 00:06:08,2015-09-01 00:00:01,2015-09-01 00:06:08,3119,3118,0,0
2015-09-01 00:15:34,2015-09-01 00:00:07,2015-09-01 00:15:34,536,340,0,0
2015-09-01 00:11:07,2015-09-01 00:00:09,2015-09-01 00:11:07,347,483,0,0


In [133]:
# Create two sub dataframes, grouping on and aggregating start data and then stop data
grouped_by = df.groupby(['start_period', 'start station id'])
df_start = pd.DataFrame({'started_count' : grouped_by.size()}).reset_index()
df_start.columns = ["period", "station_id", "start_count"]

grouped_by = df.groupby(['stop_period', 'end station id'])
df_stop = pd.DataFrame({'stopped_count' : grouped_by.size()}).reset_index()
df_stop.columns = ["period", "station_id", "stop_count"]

In [125]:
df_start.head()

,period,station_id,start_count
0,0,173,1
1,0,263,1
2,0,274,1
3,0,285,1
4,0,307,1


In [126]:
df_stop.head()

,period,station_id,stop_count
0,0,307,1
1,0,340,1
2,0,449,1
3,0,483,1
4,0,3118,1


In [95]:
# Combine these two dataframes and fill N/A values to 0
df_combined = pd.concat([df_start, df_stop])
df_combined = df_combined.fillna(0)

In [105]:
df_combined.head()

,period,start_count,station_id,stop_count
0,0,1,173,0
1,0,1,263,0
2,0,1,274,0
3,0,1,285,0
4,0,1,307,0


In [129]:
import numpy as np

# Let's group on period and station_id and then sum up along start_count and stop_count
# And reset the multi-level index so we have a normal DataFrame
aggregate_data = df_combined.groupby(['period', 'station_id']).sum().reset_index()

In [131]:
aggregate_data[:30]

,period,station_id,start_count,stop_count
0,0,173,1,0
1,0,263,1,0
2,0,274,1,0
3,0,285,1,0
4,0,307,1,1
5,0,316,1,0
6,0,317,1,0
7,0,326,1,0
8,0,340,0,1
9,0,347,1,0


# Transformation and Modeling
1. Get all unique station ids
2. For each station id, create 2 columns, e.g. 273_in 273_out
3. Translate data in `aggregate_data` into dataframe of the above format, where index=period
4. Given the feature data (in out for each station and the period), compute the target (1 if in the next period more than 1 in, 0 otherwise)
5. Run through RandomForest and measure accuracy and type 1 and type 2 errors.



In [233]:
# Let's construct a pivot table
pivoted = pd.pivot_table(
    aggregate_data,
    index=["period"],
    columns=["station_id"],
    aggfunc=np.sum,
    fill_value=0)

In [243]:
pivoted

start_count                                              ...   \
station_id        72   79   82   83   116  119  120  127  128  137  ...    
period                                                              ...    
0                    0    0    0    0    0    0    0    0    0    0 ...    
1                    0    0    0    0    0    0    0    0    0    0 ...    
2                    0    0    0    0    0    0    0    0    0    0 ...    
3                    0    0    0    0    0    0    0    0    0    0 ...    
4                    0    0    0    0    0    0    0    1    1    0 ...    
5                    0    0    0    0    0    0    0    0    1    0 ...    
6                    0    0    0    0    0    0    0    0    0    0 ...    
7                    0    0    0    1    0    0    0    0    0    0 ...    
8                    0    0    0    0    1    0    0    0    0    0 ...    
9                    0    0    0    0    1    0    0    0    0    0 ...    
10                   0    0    0    0    0    0    0    0    0    0 ...    
11                   1    0    0    0    1    0    0    0    0    0 ...    
12                   0    0    0    0    0    0    0    0    0    0 ...    
13                   0    0    0    0    0    0    0    0    0    1 ...    
14                   0    0    0    0    0    0    0    0    0    0 ...    
15                   0    0    0    0    0    0    0    0    0    0 ...    
16                   0    0    0    0    0    0    0    1    0    0 ...    
17                   0    0    0    0    0    0    0    0    0    0 ...    
18                   0    0    0    0    0    0    0    0    0    0 ...    
19                   0    0    0    0    0    0    0    0    0    0 ...    
20                   0    0    0    0    0    0    0    0    0    0 ...    
21                   0    0    0    0    1    0    0    0    0    0 ...    
22                   0    0    0    0    0    0    0    0    0    0 ...    
23                   0    0    0    0    0    0    0    0    0    0 ...    
24                   0    0    0    0    1    0    0    0    0    1 ...    
25                   0    0    0    0    0    0    0    0    0    0 ...    
26                   0    0    0    0    0    0    0    0    0    0 ...    
27                   0    0    0    0    0    0    0    0    0    0 ...    
28                   0    0    0    0    0    0    0    0    0    0 ...    
29                   0    0    0    0    0    0    0    0    0    0 ...    
...                ...  ...  ...  ...  ...  ...  ...  ...  ...  ... ...    
110                  0    1    1    0    2    0    0    2    4    0 ...    
111                  3    1    0    0    2    0    0    7    6    0 ...    
112                  0    1    1    0    2    1    0    2    1    0 ...    
113                  0    0    0    0    0    0    0    0    0    0 ...    
114                  0    0    0    0    0    0    0    0    0    0 ...    
115                  0    0    0    0    0    0    0    0    0    0 ...    
116                  0    0    0    0    0    0    0    0    0    0 ...    
117                  0    0    0    0    0    0    0    0    0    0 ...    
118                  0    0    0    0    0    0    0    0    0    0 ...    
120                  0    0    0    0    0    0    0    0    0    0 ...    
123                  0    0    0    0    0    0    0    0    0    0 ...    
125                  0    0    0    0    0    0    0    0    0    0 ...    
126                  0    0    0    0    0    0    0    0    0    0 ...    
132                  0    0    0    0    0    0    0    0    0    0 ...    
133                  0    0    0    0    0    0    0    0    0    0 ...    
136                  0    0    0    0    0    0    0    0    0    0 ...    
137                  0    0    0    0    0    0    0    0    0    0 ...    
141                  0    0    0    0    0    0    0    0    0    0 ...    
147                  0    0    0    0    0    0    0    0  

In [214]:
# Go from multilevel index of start > station_id and stop > station_id to {station_id}_start and {station_id}_stop
deeper_cols = pivoted.columns.get_level_values(1)
top_level_cols = pivoted.columns.get_level_values(0)

In [251]:
# Flattening the columns
resultant_cols = []    
for i, station_id in enumerate(deeper_cols):
    if top_level_cols[i] == "start_count":
        resultant_cols.append("{}_{}".format(station_id, "out"))
    else:
        resultant_cols.append("{}_{}".format(station_id, "in"))
pivoted.columns = resultant_cols

In [257]:
pivoted = pivoted.reset_index()

In [258]:
pivoted

,period,72_out,79_out,82_out,83_out,116_out,119_out,120_out,127_out,128_out,...,3129_in,3130_in,3133_in,3158_in,3163_in,3180_in,3182_in,3222_in,3223_in,3224_in
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3


In [284]:
TARGET_STATION_ID = "529"
X = pivoted.copy()
# Turn number of out in station TARGET_STATION_ID into 1s and 0s
answer_series = (
    pivoted["{}_out".format(TARGET_STATION_ID) ] > 1).apply(int)

In [289]:
X = X[:-1]
y = answer_series[1:]

In [290]:
len(X)

139

In [291]:
len(y)

139

In [296]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation

classifier = RandomForestClassifier()
scores = cross_validation.cross_val_score(
    classifier,
    X,
    y,
    cv=10
)

In [297]:
scores

array([ 0.73333333,  0.8       ,  0.93333333,  1.        ,  1.        ,
        1.        ,  0.69230769,  0.69230769,  1.        ,  0.84615385])